In [1]:
from account import * # keys and tokkens
import tweepy
from textblob import TextBlob

In [2]:
#setup

auth = tweepy.OAuthHandler(ConsumerKey, ConsumerSecret) 
auth.set_access_token(Access_Token, Access_Token_Secret)
api = tweepy.API(auth) 




In [3]:
#get the top trends in israel


import pandas as pd 

locations = api.trends_available()

df =pd.DataFrame(locations)

israel = df.loc[df['country'] == 'Israel']

woeid = list(israel['woeid'])


def get_trend(x=10):

    df = pd.DataFrame()

    for  i in woeid:
        trends = api.trends_place(i)[0]
        trends = trends['trends']
        A = pd.DataFrame(trends)
        df = df.append(A ,ignore_index=True)



    return df.sort_values('tweet_volume', ascending=False).drop_duplicates().reset_index(drop=True).head(x)


get_trend(15)

,name,promoted_content,query,tweet_volume,url
0,England,None,England,191372.0,http://twitter.com/search?q=England
1,White House,None,%22White+House%22,170115.0,http://twitter.com/search?q=%22White+House%22
2,Sara,None,Sara,119823.0,http://twitter.com/search?q=Sara
3,Germany,None,Germany,99081.0,http://twitter.com/search?q=Germany
4,#Iran,None,%23Iran,82752.0,http://twitter.com/search?q=%23Iran
5,#bitcoin,None,%23bitcoin,80351.0,http://twitter.com/search?q=%23bitcoin
6,Jews,None,Jews,49549.0,http://twitter.com/search?q=Jews
7,Palestinian,None,Palestinian,40258.0,http://twitter.com/search?q=Palestinian
8,#Startup,None,%23Startup,31064.0,http://twitter.com/search?q=%23Startup
9,#Erdogan,None,%23Erdogan,21361.0,http://twitter.com/search?q=%23Erdogan


In [7]:
#clean tweets
# improveEEEEE into1 regex
import re


def clean_tweet(sentence):
    a= re.sub(r'(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', '', sentence)
    sentence = re.sub(r'\w*@\w* ',' ', a)
    
    return sentence


#clean_tweet(t[0][0])

In [8]:
#transform html file to list of twitters 
# html file with info comes from the KnessetMembersTwitterAccounts1.0 scraper i made using selenium 


import pandas as pd 

df = pd.read_html('KnessetMembersTwitter.html')

Xks_info = df[0]

accounts = Xks_info[Xks_info['twitter'] != 'none']

accounts = list(accounts['twitter'])


#change twitter urls to format @ABCD
##---improve
for i in range(len(accounts)) :
   
    accounts[i] = '@'+ accounts[i].replace('https://twitter.com/','').replace('?lang=he','').replace('http://twitter.com/#!/','').replace('http://twitter.com/','').replace('ttps://twitter.com/','')
    


In [11]:
# array of tweets for each account in list 

def get_tweets(screenName,num_items):


    all_accounts = []
    

    for i in screenName[0:10] :  #only first 10 members for development
        accounts_tweets = []
        for tweet in tweepy.Cursor(api.user_timeline,screen_name = i).items(num_items):
            #text= tweet.text
            text= clean_tweet(tweet.text)
            #print(tweet.favorite_count)
            #print(tweet.in_reply_to_user_id_str)
            #print(tweet.author.screen_name)
            #print('----')
            accounts_tweets.append(text)
            
        all_accounts.append(accounts_tweets)

    return all_accounts


screenName = accounts # the accounts to retrieve tweets from 
num_items = 2 # num of tweets for each account 
tweets= get_tweets(accounts, num_items)
tweets

[['עם שגריר האיחוד האירופי בישראל, עמנואל ז׳ופרה. אמרתי לו שההצהרות האנטי ישראליות של חלק מהמנהיגים האירופים פוגעות בי… ',
  'RT @YuliEdelstein: קשה לי להאמין בקיומו של יהודי אחד שיסרב לשתות מיין שייצרו יהודים בני העדה האתיופית. גזענות מבישה. '],
 ['RT @radiodarom97: ח"כ  במתקפה קשה על מערכת המשפט בראיון לתכניתנו בוקר טוב דרום: "למערכת המשפט עלה השתן לראש, היא החליטה לנהל את ה…',
  'RT @ramonhaim: בכל פעם שהחלטתי בניגוד לדעתם איימו עלי "שבגץ  יפסול" או"אותך ליועמ"ש", מה שמעולם  לא קרה.\nדוגמה קלסית לסמכות ללא\nאחריות, ואח…'],
 ['הארץ או ארץ נהדרת? עיתון הארץ קורא למינויו של ח״כ איימן עודה, יו״ר הרשימה המשותפת לראש האופוזיציה בהיותו כ״בעד שלום… ',
  'I recently wrote to Charles Krauthammer: “I am one of the millions you’ve inspired. Your handicaps never inhibited… '],
 ['מסיבת סיום בגן טרום חובה. אבא גאה 😍😇☺️❤️ ',
  'אין לי בעיה עם ביקורת, גם עם הקשה ביותר. אבל אין שום סיבה לציין בשידור חי את ארץ המוצא שלי בהקשר להצעת חוק. '],
 ['בושה וחרפה שהגזענות ככה מרימה ראשה. חרם צרכנים בודאי ישנה

In [12]:
##### input tweets array with each members array from get_tweets
##### output word count for each member

def WWWords (tweets) :   
    m= []
    for i in range(len(tweets)) :
        
        m.append( word_count(listOfTweetsToWords(tweets[i])))
        
    return countsArray


#wordCountsArray = WWWords(tweets)


In [13]:
def listOfTweetsToWords(tweetList):


    word_list = ''.join(tweetList)

    word_list = word_list.split()
    
    return word_list


def word_count(tweets):
    
    if type(tweets) is str :
        word_list = tweets.split()
        
    else:
        word_list = tweets

    word_count={}

    for word in word_list:
        word_count[word] = word_count.get(word,0) +1
    return word_count







In [14]:
#sentiment

def sentiment(word, lang = 'en'):

    twitts = api.search(word,lang)
    for twitt in twitts : 
        print(twitt.text)
        analysis = TextBlob(twitt.text)
        print(analysis.sentiment)
        print(twitt.author.screen_name)
        print("")

    
#sentiment('Israel')

In [15]:




def getreaction (searchTerm,noOfSearches):

    def precentage (part,whole):
        return 100 * float(part)/float(whole)



    polarity = 0
    negative = 0
    positive = 0
    neutral = 0

    tweets = tweepy.Cursor(api.search ,q= searchTerm).items(noOfSearches)

    for tweet in tweets :
        analysis = TextBlob(tweet.text)
        polarity += analysis.sentiment.polarity

        if (analysis.sentiment.polarity == 0):
            neutral += 1
        elif (analysis.sentiment.polarity > 0):
            positive += 1
        elif (analysis.sentiment.polarity < 0):
            negative += 1
        
    positive = format(precentage(positive, noOfSearches) ,'.0f')
    negative = format(precentage(negative, noOfSearches) ,'.0f')
    neutral = format(precentage(neutral, noOfSearches)  ,'.0f')    

    if (polarity == 0):
           reaction = 'neutral' 
    elif (polarity > 0):
           reaction = 'POSITIVE'
    elif (polarity < 0):
            reaction = 'NEGATIVE' 


    print ('People reaction to {}  according to {} tweets is {}'.format(searchTerm,noOfSearches,reaction))

    print()
    print(str(positive) + "% people thought it was positive")
    print(str(negative) + "% people thought it was negative")
    print(str(neutral) + "% people thought it was neutral")

    
word = 'israel'
num = 100

#getreaction(word,num)


In [ ]:
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

In [ ]:
#textBlob
import nltk

word_list = t[0]
tweet = ' '.join(word for word in word_list)
txt = TextBlob(tweet)
tags = txt.tags
word_counts = txt.word_counts
noun = txt.noun_phrases
sent = txt.sentences